In [96]:
'''
计算一次实验中的多路iFEMG信号的特征

更改4通道iFEMG采集板
1600采样率

没有计算主动与拮抗肌肉之间的差值'''

import pandas as pd

from utils_gui_model import form_sbj_info_df, form_feature_df_1ch
from utils_iFEMG_feature import df_save_csv

In [99]:
"""
init dataframe
"""
feature_name_list = ['initial_pressure_ave', 'FMG', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']
sbj_info_name_list = ['subject_name', 'height', 'weight', 'age', 'gender', 'label']
sbj_info_col1_list = ['subject_info' for i in sbj_info_name_list]

ch1_str = "bicps_br"
ch2_str = "tricps_br_medial"
ch3_str = "tricps_br_lateral"

ch1_name_list = [ch1_str for i in feature_name_list]
ch2_name_list = [ch2_str for i in feature_name_list]
ch3_name_list = [ch3_str for i in feature_name_list]

feature_df = pd.DataFrame(columns=pd.MultiIndex.from_product([[ch1_str, ch2_str, ch3_str], feature_name_list]))
sbj_info_df = pd.DataFrame(columns=pd.MultiIndex.from_product([['subject_info'], sbj_info_name_list]))
all_feature_df = pd.concat([sbj_info_df, feature_df], axis = 1)
print(all_feature_df)
print("Init dataframe shape: ", all_feature_df.shape)

Empty DataFrame
Columns: [(subject_info, subject_name), (subject_info, height), (subject_info, weight), (subject_info, age), (subject_info, gender), (subject_info, label), (bicps_br, initial_pressure_ave), (bicps_br, FMG), (bicps_br, mav), (bicps_br, rms), (bicps_br, wave_length), (bicps_br, zero_crossing), (bicps_br, slope_sign_change), (bicps_br, mean_freq), (bicps_br, mean_power_freq), (tricps_br_medial, initial_pressure_ave), (tricps_br_medial, FMG), (tricps_br_medial, mav), (tricps_br_medial, rms), (tricps_br_medial, wave_length), (tricps_br_medial, zero_crossing), (tricps_br_medial, slope_sign_change), (tricps_br_medial, mean_freq), (tricps_br_medial, mean_power_freq), (tricps_br_lateral, initial_pressure_ave), (tricps_br_lateral, FMG), (tricps_br_lateral, mav), (tricps_br_lateral, rms), (tricps_br_lateral, wave_length), (tricps_br_lateral, zero_crossing), (tricps_br_lateral, slope_sign_change), (tricps_br_lateral, mean_freq), (tricps_br_lateral, mean_power_freq)]
Index: []

[0 r

In [100]:
# 存储文件名对应重量标签的dic
bicps_dic = {'bi-0': 0, 'bi-05': 0.5, 'bi-1': 1, 'bi-2': 2}
tricps_dic = {'tri-0': 0, 'tri-05': 0.5, 'tri-1': 1}

# 需要更改
# 更换受试需要更改
signal_channel_dic = {"bicps_br": 1,
                      "tricps_br_lateral": 3,
                      "tricps_br_medial": 4}
folder_path = r"E:\Data\paper2\iFEMG\zk-2"
subject_name = "zk-2"
subject_args_dict = {"subject_name": subject_name,
                     "height": 178,
                    "weight": 66,
                    "age": 22,
                    "gender": 1}
fs = 1222   # 1626 or 1222
channel_nums = 8
# 更换动作需要更改
file_name_dic = tricps_dic

###########################################################################
# 以上都需要改
############################################################################

# key是文件名，value是对应的重量标签
for key, value in file_name_dic.items():
    # ch1_name_list: bicps_br
    temp_fea_df1 = form_feature_df_1ch(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["bicps_br"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = fs, 
                                    ch_nums=channel_nums)
    temp_fea_df1.columns = [ch1_name_list, feature_name_list]
    # ch2_name_list: tricps_med
    temp_fea_df2 = form_feature_df_1ch(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["tricps_br_medial"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = fs,
                                    ch_nums=channel_nums)
    temp_fea_df2.columns = [ch2_name_list, feature_name_list]
    # ch3_name_list: tricps_lat
    temp_fea_df3 = form_feature_df_1ch(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["tricps_br_lateral"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = fs,
                                    ch_nums=channel_nums)
    temp_fea_df3.columns = [ch3_name_list, feature_name_list]

    # 受试者个人信息和重量label
    temp_sbjinfo_df = form_sbj_info_df(df_len = len(temp_fea_df1), **subject_args_dict, label = value)
    temp_sbjinfo_df.columns = [sbj_info_col1_list, sbj_info_name_list]
    
    # 合并df
    temp_df = pd.concat([temp_sbjinfo_df, temp_fea_df1, temp_fea_df2, temp_fea_df3], axis=1)
    print(temp_df.shape)
    all_feature_df = pd.concat([all_feature_df, temp_df], axis = 0, ignore_index=True)
    print(all_feature_df.shape)
    pass
    
all_feature_df.head


(10, 33)
(10, 33)


C:\Users\WeimyMark\AppData\Local\Temp\ipykernel_11368\2536072314.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_feature_df = pd.concat([all_feature_df, temp_df], axis = 0, ignore_index=True)


(10, 33)
(20, 33)
(10, 33)
(30, 33)


<bound method NDFrame.head of    subject_info                                            bicps_br  \
   subject_name height weight age gender label initial_pressure_ave   
0           178     66     22   1   zk-2     0          2776.603015   
1           178     66     22   1   zk-2     0          2776.603015   
2           178     66     22   1   zk-2     0          2776.603015   
3           178     66     22   1   zk-2     0          2776.603015   
4           178     66     22   1   zk-2     0          2776.603015   
5           178     66     22   1   zk-2     0          2776.603015   
6           178     66     22   1   zk-2     0          2776.603015   
7           178     66     22   1   zk-2     0          2776.603015   
8           178     66     22   1   zk-2     0          2776.603015   
9           178     66     22   1   zk-2     0          2776.603015   
10          178     66     22   1   zk-2   0.5          2668.391117   
11          178     66     22   1   zk-2   0.5 

In [101]:
"""
写入文件，增加覆盖检查功能
运行前更改：
    1. 肌肉名称
"""
filename = folder_path + '\\' + subject_name + "_extension_abs.csv"

df_save_csv(all_feature_df, filename)

File E:\Data\paper2\iFEMG\zk-2\zk-2_extension_abs.csv saved!
